# Definitions

In [1]:
%matplotlib inline
from pathlib import Path
from pprint import pprint
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 1.5})

In [2]:
import phconvert as phc
import polimi_tcspc
print(phc.__version__)

0.7.3


In [3]:
def save_polimi_tcspc_to_photon_hdf5(filename, metadata=None, validate=False):
    hardware = ('POLIMI-TCSPC module 16-CH [Pos1: 0403FE6A (ch 1-8), Pos3: 0403F16A (ch 9-16)]; '
                'Manta 0403DAE6.')
    if metadata is None:
        metadata = {}
    d = metadata.copy()
    d['description'] = '\n'.join((d.get('description', ''), hardware))
    d.setdefault('identity', {})

    fpath = Path(filename)
    h5_fname = Path(fpath.parent, '%s_%s.hdf5' % (fpath.parts[-2], fpath.stem))
    filename = str(fpath)
    d.setdefault('provenance', {'filename': filename})
    print('\n\n>>> FILE: %s\n%s' % (filename, d['description']))
    pprint(metadata)
    
    data, header_info = polimi_tcspc.loadfile(filename)
    timestamps_m = data['timestamps']
    nanotimes = data['nanotimes']
    channels = data['channels']
    timestamps_unit = 1/header_info['timestamps_clock_Hz']
    acquisition_duration = np.round((np.max([t.max() for t in timestamps_m]) - 
                                     np.min([t.min() for t in timestamps_m])) * timestamps_unit, 1)
    
    tcspc_num_bins = 2**14
    for ch in range(16):
        ch_mask = channels == ch
        tcspc_unit = header_info['tcspc_binwidths_ps'][ch]*1e-12

        photon_data = dict(
            timestamps = timestamps_m[ch],
            timestamps_specs = dict(timestamps_unit=timestamps_unit),
            detectors = channels[ch_mask],

            nanotimes = nanotimes[ch_mask],

            nanotimes_specs = dict(
                tcspc_unit = tcspc_unit,
                tcspc_range = tcspc_unit*tcspc_num_bins,
                tcspc_num_bins = tcspc_num_bins))

        d['photon_data%d' % ch] = photon_data

    d['acquisition_duration'] = acquisition_duration
    d['_filename'] = filename
    if 'setup' not in d:
        d['setup'] = {}
    d['setup'].update(
        {'num_pixels': header_info['num_channels'], 
         'num_spots': header_info['num_channels'], 
         'num_spectral_ch': 1, 'num_polarization_ch': 1, 'num_split_ch': 1,
         'modulated_excitation': False, 'lifetime': True,
         'excitation_wavelengths': [532e-9], 'excitation_cw': [False],
         'detection_wavelengths': [580e-9],
         #'excitation_alternated': (False,),  # uncomment for Photon-HDF5 0.5+
         'user': {'name': hardware}}
    )
    phc.hdf5.save_photon_hdf5(d, h5_fname=str(h5_fname), overwrite=True, validate=validate, skip_measurement_specs=False)

# Check FIFO-full flags

In [4]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-09-04'

In [5]:
for file in Path(basepath).glob('**/*.dat'):
    if not polimi_tcspc.is_polimi_tcspc(str(file)): 
        continue

    print('\n - %s' % Path(*file.parts[3:]), flush=True)
    fifo_full = polimi_tcspc.get_fifo_full_array(str(file))
    
    if fifo_full.any():
        print('     * FIFO Full!', flush=True)


 - data\POLIMI-TCSPC-16ch\2015-09-04\IRF-glass-air-100mW-BP-Dich532-635_10.dat
     * FIFO Full!

 - data\POLIMI-TCSPC-16ch\2015-09-04\IRF-glass-air-100mW-BP-Dich532-635_9.dat

 - data\POLIMI-TCSPC-16ch\2015-09-04\IRF-glass-air-100mW-LP-BP-Dich532-635_8.dat

 - data\POLIMI-TCSPC-16ch\2015-09-04\IRF-glass-water-surface-100mW-BP-Dich532-635_11.dat

 - data\POLIMI-TCSPC-16ch\2015-09-04\irf-H2O-50um-100mW-BP-Dich532-635_2.dat

 - data\POLIMI-TCSPC-16ch\2015-09-04\irf-H2O-50um-100mW-LP-BP_Dich532-635_1.dat

 - data\POLIMI-TCSPC-16ch\2015-09-04\T-NTD-3new_66pM-100mW-LP-BP-Dich532-635_7.dat

 - data\POLIMI-TCSPC-16ch\2015-09-04\T-NTD-3_200pM-100mW-LP-BP-Dich532-635_3.dat

 - data\POLIMI-TCSPC-16ch\2015-09-04\T-NTD-3_50pM-100mW-LP-BP-Dich532-635_4.dat


In [6]:
metapath = Path(basepath).joinpath('metadata.yaml')
metapath.exists()

True

In [7]:
metadata = yaml.load(metapath.open())

In [8]:
print(yaml.dump(metadata, default_flow_style=False))

IRF-glass-air-100mW-BP-Dich532-635_10:
  description: 'Test IRF

    '
  setup: &id001
    excitation_input_powers:
    - 0.1
IRF-glass-air-100mW-BP-Dich532-635_9:
  description: 'Test IRF

    '
  setup: *id001
IRF-glass-air-100mW-LP-BP-Dich532-635_8:
  description: 'Test IRF

    '
  setup: *id001
IRF-glass-water-surface-100mW-BP-Dich532-635_11:
  description: 'Good IRF with high SNR. Acquired focusing on the glass-water surface
    of a water sample. Filters: only 580-band-pass.

    '
  setup: *id001
T-NTD-3_200pM-100mW-LP-BP-Dich532-635_3:
  description: 'Measurement with wrong Z position of SPAD (along the optical axis).

    '
  sample: &id002
    buffer_name: TE50
    dye_names: ATTO550
    num_dyes: 1
    sample_name: 80bp dsDNA labeled with ATTO550, T-NTD(-3)
  setup: *id001
T-NTD-3_50pM-100mW-LP-BP-Dich532-635_4:
  description: 'Measurement with wrong Z position of SPAD (along the optical axis).

    '
  sample: *id002
  setup: *id001
T-NTD-3new_66pM-100mW-LP-BP-Dich532-635_

In [9]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-09-08'

In [10]:
for file in Path(basepath).glob('**/*.dat'):
    if not polimi_tcspc.is_polimi_tcspc(str(file)): 
        continue

    print('\n - %s' % Path(*file.parts[3:]), flush=True)
    fifo_full = polimi_tcspc.get_fifo_full_array(str(file))
    
    if fifo_full.any():
        print('     * FIFO Full!', flush=True)


 - data\POLIMI-TCSPC-16ch\2015-09-08\DA_12_D_8_MIX_LP-BP-100mW_6.dat

 - data\POLIMI-TCSPC-16ch\2015-09-08\IRF-TE50-surface-BP-100mW_3.dat

 - data\POLIMI-TCSPC-16ch\2015-09-08\IRF-TE50-surface-plus-BP-100mW_4.dat

 - data\POLIMI-TCSPC-16ch\2015-09-08\T-NTD-3-66pM-60uL-100mW_2.dat

 - data\POLIMI-TCSPC-16ch\2015-09-08\TA-15-NT-3_LP-BP-100mW_5.dat


In [11]:
metapath = Path(basepath).joinpath('metadata.yaml')
metapath.exists()

True

In [12]:
metadata = yaml.load(metapath.open(),)

In [13]:
print(yaml.dump(metadata, default_flow_style=False))

DA_12_D_8_MIX_LP-BP-100mW_6:
  description: 'Mixture of 60% FRET, 40% DO sample, 60uL, 66pM (total). No gasket.

    '
  sample:
    buffer_name: TE50
    dye_names: ATTO550, ATTO647N
    num_dyes: 2
    sample_name: Mixture 60% FRET TA(-15)-NTD(-3), 40% DO T-NTD(-3)80bp, 66pM total.
  setup: &id001
    excitation_input_powers:
    - 0.1
IRF-TE50-surface-BP-100mW_3:
  description: 'IRF measurement

    '
  setup: *id001
IRF-TE50-surface-plus-BP-100mW_4:
  description: 'IRF measurement

    '
  setup: *id001
T-NTD-3-66pM-60uL-100mW_2:
  description: ''
  sample: &id002
    buffer_name: TE50
    dye_names: ATTO550
    num_dyes: 1
    sample_name: 80bp dsDNA labeled with ATTO550, T-NTD(-3)
  setup: *id001
TA-15-NT-3_LP-BP-100mW_5:
  description: 'Medium FRET sample 20uL no gasket.

    '
  sample: &id003
    buffer_name: TE50
    dye_names: ATTO550, ATTO647N
    num_dyes: 2
    sample_name: 80bp dsDNA, TA(-15)-NTD(-3)
  setup: *id001
sample_d: *id002
sample_da: *id003
setup: *id001



In [14]:
metadata

{'DA_12_D_8_MIX_LP-BP-100mW_6': {'description': 'Mixture of 60% FRET, 40% DO sample, 60uL, 66pM (total). No gasket.\n',
  'sample': {'buffer_name': 'TE50',
   'dye_names': 'ATTO550, ATTO647N',
   'num_dyes': 2,
   'sample_name': 'Mixture 60% FRET TA(-15)-NTD(-3), 40% DO T-NTD(-3)80bp, 66pM total.'},
  'setup': {'excitation_input_powers': [0.1]}},
 'IRF-TE50-surface-BP-100mW_3': {'description': 'IRF measurement\n',
  'setup': {'excitation_input_powers': [0.1]}},
 'IRF-TE50-surface-plus-BP-100mW_4': {'description': 'IRF measurement\n',
  'setup': {'excitation_input_powers': [0.1]}},
 'T-NTD-3-66pM-60uL-100mW_2': {'description': '',
  'sample': {'buffer_name': 'TE50',
   'dye_names': 'ATTO550',
   'num_dyes': 1,
   'sample_name': '80bp dsDNA labeled with ATTO550, T-NTD(-3)'},
  'setup': {'excitation_input_powers': [0.1]}},
 'TA-15-NT-3_LP-BP-100mW_5': {'description': 'Medium FRET sample 20uL no gasket.\n',
  'sample': {'buffer_name': 'TE50',
   'dye_names': 'ATTO550, ATTO647N',
   'num_dy

# Convert files

## 2015-09-04  Measurements

In [15]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-09-04'

In [16]:
filenames = ['T-NTD-3new_66pM-100mW-LP-BP-Dich532-635_7.dat',
             'IRF-glass-air-100mW-BP-Dich532-635_10.dat',
             'IRF-glass-water-surface-100mW-BP-Dich532-635_11.dat',
            ]

In [17]:
for filename in filenames:
    datafile = Path(basepath, filename)
    assert datafile.exists()

    metapath = Path(basepath, 'metadata.yaml')
    metadata_all = yaml.load(metapath.open())
    assert datafile.stem in metadata_all
    metadata = metadata_all[datafile.stem]
    #pprint(metadata)
    
    save_polimi_tcspc_to_photon_hdf5(datafile, metadata=metadata, validate=True)



>>> FILE: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-09-04\T-NTD-3new_66pM-100mW-LP-BP-Dich532-635_7.dat
Good measurement of 60uL of sample without gasket. 20uL of buffer were added within 30s of measurement start  to reduce concentration from 100pM to 66pM.

POLIMI-TCSPC module 16-CH [Pos1: 0403FE6A (ch 1-8), Pos3: 0403F16A (ch 9-16)]; Manta 0403DAE6.
{'description': 'Good measurement of 60uL of sample without gasket. 20uL of '
                'buffer were added within 30s of measurement start  to reduce '
                'concentration from 100pM to 66pM.\n',
 'sample': {'buffer_name': 'TE50',
            'dye_names': 'ATTO550',
            'num_dyes': 1,
            'sample_name': '80bp dsDNA labeled with ATTO550, T-NTD(-3)'},
 'setup': {'excitation_input_powers': [0.1]}}
Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-09-04\2015-09-04_T-NTD-3new_66pM-100mW-LP-BP-Dich532-635_7.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field 

## 2015-09-08  Measurements

In [18]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-09-08'

In [19]:
filenames = ['TA-15-NT-3_LP-BP-100mW_5.dat',
             'DA_12_D_8_MIX_LP-BP-100mW_6.dat',
             'IRF-TE50-surface-BP-100mW_3.dat',
             'IRF-TE50-surface-plus-BP-100mW_4.dat'
            ]

In [20]:
for filename in filenames:
    datafile = Path(basepath, filename)
    assert datafile.exists()

    metapath = Path(basepath, 'metadata.yaml')
    metadata_all = yaml.load(metapath.open())
    assert datafile.stem in metadata_all
    metadata = metadata_all[datafile.stem]
    #pprint(metadata)
    
    save_polimi_tcspc_to_photon_hdf5(datafile, metadata=metadata, validate=True)



>>> FILE: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-09-08\TA-15-NT-3_LP-BP-100mW_5.dat
Medium FRET sample 20uL no gasket.

POLIMI-TCSPC module 16-CH [Pos1: 0403FE6A (ch 1-8), Pos3: 0403F16A (ch 9-16)]; Manta 0403DAE6.
{'description': 'Medium FRET sample 20uL no gasket.\n',
 'sample': {'buffer_name': 'TE50',
            'dye_names': 'ATTO550, ATTO647N',
            'num_dyes': 2,
            'sample_name': '80bp dsDNA, TA(-15)-NTD(-3)'},
 'setup': {'excitation_input_powers': [0.1]}}
Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-09-08\2015-09-08_TA-15-NT-3_LP-BP-100mW_5.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field "author_affiliation" in "/identity".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data10".
Photon-HDF5 WARNING: Missing field "measurement_spec